In [42]:
class CharacterClass(object):
    
    def __init__(self, _class):
        self._class = _class.lower()
        
    def __str__(self):
        return self._class

In [45]:
class CharacterRace(object):
    
    def __init__(self, race):
        self._race = race.lower()
        
    def __str__(self):
        return self._race

In [46]:
class Player(object):
    
    def __init__(self, name, race, _class, ability_scores, level=1):
        self.name = name
        self._race = CharacterRace(race)
        self._class = CharacterClass(_class)
        self.level = level
        self.__spellcaster = str(self._class) in [
            "bard", "druid", "ranger", "sorceror", "wizard", "warlock"]
        self.ability_scores = {
            "Strength" : ability_scores[0],
            "Dexterity" : ability_scores[1],
            "Constitution" : ability_scores[2],
            "Intelligence" : ability_scores[3],
            "Wisdom" : ability_scores[4],
            "Charisma" : ability_scores[5]
        }
        
    def __str__(self):
        return "A level {0} {1} {2} called {3}".format(
            self.level, 
            str(self._race).title(),
            str(self._class).title(),
            self.name.title())
    
    def attack(self):
        pass

In [47]:
player1 = Player("Merret Strongheart",
                   "Halfling",
                   "Ranger",
                   [12, 20, 12, 8, 16, 8],
                   9)

In [48]:
print(player1._class)

ranger


In [49]:
player1._Player__spellcaster

True

In [50]:
print(player1)

A level 9 Halfling Ranger called Merret Strongheart


In [51]:
player2 = Player("Despair", "Tiefling", "Sorceror", [9, 11, 15, 14, 13, 20], 8)

In [52]:
player2._Player__spellcaster

True

In [53]:
print(player2)

A level 8 Tiefling Sorceror called Despair
